In [1]:
import json
import re
import os


# only needed if trailing object error

# for filename in os.listdir('../logs/linux_jonas/vanilla_plotly_logs/'):
#     f = os.path.join('../logs/linux_jonas/vanilla_plotly_logs/', filename)
#     if os.path.isfile(f) & f.endswith(".json"):
#         with open(f, 'r') as r:
#             content = r.read()
#         content = re.sub(r'}\s*{', '},{', content)
#         json_data = json.loads('[' + content + ']')
#         with open(f, 'w') as w:
#             json.dump(json_data, w)

In [2]:
import pandas as pd
import numpy as np
import json
import os

# df = pd.read_json('logs/n100000_s100_t10_everynth.json', orient ='index')
df = pd.DataFrame()


for filename in os.listdir('../logs/final/vanilla_pr_logs/'):
    f = os.path.join('../logs/final/vanilla_pr_logs/', filename)
    if os.path.isfile(f)  & (filename.split('.')[1] == 'json'):
        dft = pd.read_json(f)
        dft['datapoints'] = filename.split('_')[0][1:]
        dft['samples'] = filename.split('_')[1][1:]
        dft['traces'] = filename.split('_')[2][1:]
        dft['aggregator'] = filename.split('_')[3]
        dft['iteration'] = filename.split("_")[4].split('.')[0][-1:]
        df = pd.concat([df, dft], ignore_index=True)
        


df['message']= df['message'].str.split('\"').str[-2]
df.drop(df[df['level'].eq('WARNING')].index, inplace=True)
df = df.drop('source', axis=1)
df.reset_index()
df['message']


0                                           100.0%
1             time (visible): 1805.199951171875 ms
2        render time (visible): 827.38818359375 ms
3              time (invisible): 1.220947265625 ms
4                time (full): 1808.047119140625 ms
                           ...                    
65995          time (visible): 261.415771484375 ms
65996      render time (visible): 81.3662109375 ms
65997          time (invisible): 0.520751953125 ms
65998             time (full): 264.547607421875 ms
65999                render time (invisible): None
Name: message, Length: 66000, dtype: object

In [3]:
df_prio = pd.DataFrame()


for filename in os.listdir('../logs/final/visual_gain_logs/'):
    f = os.path.join('../logs/final/visual_gain_logs/', filename)
    if os.path.isfile(f) & f.endswith(".json"):
        dft = pd.read_json(f)
        dft['datapoints'] = filename.split('_')[0][1:]
        dft['samples'] = filename.split('_')[1][1:]
        dft['traces'] = filename.split('_')[2][1:]
        dft['aggregator'] = filename.split('_')[3]
        dft['iteration'] = filename.split("_")[4].split('.')[0][-1:]
        df_prio = pd.concat([df_prio, dft], ignore_index=True)
        
df_prio['message']= df_prio['message'].str.split('\"').str[-2]
df_prio.drop(df_prio[df_prio['level'].eq('SEVERE')].index, inplace=True)
df_prio.drop('source', axis=1, inplace=True)
df_prio.reset_index()
df_prio

,level,message,timestamp,datapoints,samples,traces,aggregator,iteration
0,INFO,100.0%,2023-08-11 17:04:16.205,10000000,10000,100,everynth,0
1,DEBUG,time (visible): 1942.76708984375 ms,2023-08-11 17:04:19.234,10000000,10000,100,everynth,0
2,DEBUG,render time (visible): 942.869140625 ms,2023-08-11 17:04:19.236,10000000,10000,100,everynth,0
3,DEBUG,time (invisible): 1.306884765625 ms,2023-08-11 17:04:19.237,10000000,10000,100,everynth,0
4,DEBUG,time (full): 1945.81103515625 ms,2023-08-11 17:04:19.237,10000000,10000,100,everynth,0
...,...,...,...,...,...,...,...,...
65997,DEBUG,time (visible): 168.304931640625 ms,2023-08-12 00:45:01.208,100000,5000,50,everynth,9
65998,DEBUG,render time (visible): 67.005126953125 ms,2023-08-12 00:45:01.208,100000,5000,50,everynth,9
65999,DEBUG,time (invisible): 220.11083984375 ms,2023-08-12 00:45:01.428,100000,5000,50,everynth,9
66000,DEBUG,render time (invisible): 72.118896484375 ms,2023-08-12 00:45:01.429,100000,5000,50,everynth,9


In [4]:
mask = df['message'].str.contains('%')
percentage = df[mask]
percentage
df.loc[mask, 'percentage'] = percentage['message']
df['percentage'].fillna(method='ffill', inplace=True)
df.drop(percentage.index, inplace=True)
df = df.reset_index()
df[['message', 'percentage','timestamp']]

,message,percentage,timestamp
0,time (visible): 1805.199951171875 ms,100.0%,2023-08-07 17:06:47.462
1,render time (visible): 827.38818359375 ms,100.0%,2023-08-07 17:06:47.463
2,time (invisible): 1.220947265625 ms,100.0%,2023-08-07 17:06:47.464
3,time (full): 1808.047119140625 ms,100.0%,2023-08-07 17:06:47.464
4,render time (invisible): None,100.0%,2023-08-07 17:06:47.464
...,...,...,...
59995,time (visible): 261.415771484375 ms,10.0%,2023-08-08 00:53:52.332
59996,render time (visible): 81.3662109375 ms,10.0%,2023-08-08 00:53:52.332
59997,time (invisible): 0.520751953125 ms,10.0%,2023-08-08 00:53:52.332
59998,time (full): 264.547607421875 ms,10.0%,2023-08-08 00:53:52.333


In [5]:
mask = df_prio['message'].str.contains('%')
percentage = df_prio[mask]
percentage
df_prio.loc[mask, 'percentage'] = percentage['message']
df_prio['percentage'].fillna(method='ffill', inplace=True)
df_prio.drop(percentage.index, inplace=True)
df_prio = df_prio.reset_index()
df_prio[['message', 'percentage','timestamp']]

,message,percentage,timestamp
0,time (visible): 1942.76708984375 ms,100.0%,2023-08-11 17:04:19.234
1,render time (visible): 942.869140625 ms,100.0%,2023-08-11 17:04:19.236
2,time (invisible): 1.306884765625 ms,100.0%,2023-08-11 17:04:19.237
3,time (full): 1945.81103515625 ms,100.0%,2023-08-11 17:04:19.237
4,render time (invisible): None,100.0%,2023-08-11 17:04:19.240
...,...,...,...
59995,time (visible): 168.304931640625 ms,10.0%,2023-08-12 00:45:01.208
59996,render time (visible): 67.005126953125 ms,10.0%,2023-08-12 00:45:01.208
59997,time (invisible): 220.11083984375 ms,10.0%,2023-08-12 00:45:01.428
59998,render time (invisible): 72.118896484375 ms,10.0%,2023-08-12 00:45:01.429


In [6]:
df['type']= df['message'].str.split().str[0]
df['update'] = df['message'].str.split("(").str[1].str.split(")").str[0]
df['time (ms)'] = df['message'].str.split(":").str[1].str.split().str[0]
df['type'] = df['type'].apply(lambda x: x + ' time' if x == 'render' else x)

In [7]:
df_prio['type']= df_prio['message'].str.split().str[0]
df_prio['update'] = df_prio['message'].str.split("(").str[1].str.split(")").str[0]
df_prio['time (ms)'] = df_prio['message'].str.split(":").str[1].str.split().str[0]
df_prio['type'] = df_prio['type'].apply(lambda x: x + ' time' if x == 'render' else x)
df_prio[['time (ms)', 'update','type']]

,time (ms),update,type
0,1942.76708984375,visible,time
1,942.869140625,visible,render time
2,1.306884765625,invisible,time
3,1945.81103515625,full,time
4,None,invisible,render time
...,...,...,...
59995,168.304931640625,visible,time
59996,67.005126953125,visible,render time
59997,220.11083984375,invisible,time
59998,72.118896484375,invisible,render time


In [13]:
from natsort import natsort_keygen

# Convert "time (ms)" column to numeric data type
df['time (ms)'] = pd.to_numeric(df['time (ms)'], errors='coerce')
df['traces'] = pd.to_numeric(df['traces'])

# aggregation_functions = {
#     'time (ms)': ['mean', lambda x: np.std(x, ddof=0) if len(x) > 1 else 0, 'var']
# }
aggregation_functions = {
    'time (ms)': ['mean', 'var']
}
sample_size = 25

filtered_df = df.query('`time (ms)` == `time (ms)`')
filtered_df_visible = filtered_df[filtered_df['update']=='visible']
filtered_df_visible = filtered_df_visible[filtered_df_visible['type']=='time']
filtered_df_visible = filtered_df_visible[filtered_df_visible['percentage'] == "100.0%"]

grouped_df = filtered_df_visible.groupby(['percentage', 'datapoints', 'samples', 'traces']).agg(aggregation_functions)
agg_df = grouped_df.reset_index()

agg_df.columns = ['percentage', 'datapoints', 'samples', 'traces', 'mean_time', 'variance']

agg_df['std_dev'] = np.sqrt(agg_df["variance"])
agg_df['std_err'] = 1.96 * agg_df["std_dev"]/np.sqrt(sample_size)

agg_df = agg_df.sort_values(by=['datapoints','samples','traces'],key=natsort_keygen())


In [14]:
# Convert "time (ms)" column to numeric data type
df_prio['time (ms)'] = pd.to_numeric(df_prio['time (ms)'], errors='coerce')
df_prio['traces'] = pd.to_numeric(df_prio['traces'])

# aggregation_functions = {
#     'time (ms)': ['mean', lambda x: np.std(x, ddof=0) if len(x) > 1 else 0, 'var']
# }
aggregation_functions = {
    'time (ms)': ['mean', 'var']
}

filtered_df_prio = df_prio.query('`time (ms)` == `time (ms)`')
filtered_df_prio_visible = filtered_df_prio[filtered_df_prio['update']=='visible']
filtered_df_prio_visible = filtered_df_prio_visible[filtered_df_prio_visible['type']=='time']

grouped_df_prio = filtered_df_prio_visible.groupby(['percentage','datapoints', 'samples', 'traces']).agg(aggregation_functions)
agg_df_prio = grouped_df_prio.reset_index()
agg_df_prio.columns = ['percentage', 'datapoints', 'samples', 'traces', 'mean_time', 'variance']
agg_df_prio['std_dev'] = np.sqrt(agg_df_prio["variance"])
agg_df_prio['std_err'] = 1.96 * agg_df_prio["std_dev"]/np.sqrt(sample_size)



agg_df_prio = agg_df_prio.sort_values(by=['datapoints','samples','traces','percentage'],key=natsort_keygen())


In [15]:
agg_df['percentage'] = '100.0% (baseline)'
df_final = pd.concat([agg_df , agg_df_prio], ignore_index=True)
df_final = df_final.sort_values(by=['datapoints','samples','traces','percentage'],key=natsort_keygen())
df_final = df_final.reset_index(drop=True)

In [11]:
df_final.to_csv('../logs/final/processed_logs_final.csv')